# Intelligence Bio Inspiree
## TP : Reseaux de neurones

#### import of libraries

In [1]:
import random
import gzip # pour décompresser les données
import pickle # pour désérialiser les données
import numpy # pour pouvoir utiliser des matrices
import matplotlib.pyplot as plt # pour l'affichage
import torch,torch.utils.data
from numpy import array, dot
import math

### Load and read train and test data

In [2]:
TRAIN_BATCH_SIZE = 1
# on charge les données de la base MNIST
data = pickle.load(gzip.open('mnist.pkl.gz'),encoding='bytes')
# images de la base d'apprentissage
train_data = torch.Tensor(data[0][0])
# labels de la base d'apprentissage
train_data_label = torch.Tensor(data[0][1])
# images de la base de test
test_data = torch.Tensor(data[1][0])
# labels de la base de test
test_data_label = torch.Tensor(data[1][1])
# on crée la base de données d'apprentissage (pour torch)
train_dataset = torch.utils.data.TensorDataset(train_data,train_data_label)
# on crée la base de données de test (pour torch)
test_dataset = torch.utils.data.TensorDataset(test_data,test_data_label)
# on crée le lecteur de la base de données d'apprentissage (pour torch)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
# on crée le lecteur de la base de données de test (pour torch)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)
# 10 fois

#### Les differentes fonctions utilisées 

In [30]:
# Fonction qui sert à initialiser les poids au debut de l'apprentissage

def Weights(num,LowerBound, UpperBound):
    w=[]
    for i in range(num): 
        w.append(random.uniform(LowerBound, UpperBound))
    
    return w 


def sigmoid(x):
    return float(1 / (1 + math.exp(-x)))


def affichage(image,label):
    # on redimensionne l'image en 28x28
    image = image.reshape(28,28)
    # on récupère à quel chiffre cela correspond (position du 1 dans label)
    label = numpy.argmax(label)
    # on crée une figure
    plt.figure()
    # affichage du chiffre
    # le paramètre interpolation='nearest' force python à afficher chaque valeur de la matrice sans l'interpoler avec ses voisines
    # le paramètre cmap définit l'échelle de couleur utilisée (ici noire et blanc)
    plt.imshow(image,interpolation='nearest',cmap='binary')
    # on met un titre
    plt.title('chiffre '+str(label))
    # on affichage les figures créées
    plt.show()
    
    
def affichage2(image,label):
    # on redimensionne l'image en 28x28
    image = image.reshape(28,28)
    # on récupère à quel chiffre cela correspond (position du 1 dans label)
    # on crée une figure
    plt.figure()
    # affichage du chiffre
    # le paramètre interpolation='nearest' force python à afficher chaque valeur de la matrice sans l'interpoler avec ses voisines
    # le paramètre cmap définit l'échelle de couleur utilisée (ici noire et blanc)
    plt.imshow(image,interpolation='nearest',cmap='binary')
    # on met un titre
    plt.title('chiffre '+str(label))
    # on affichage les figures créées
    plt.show()
    



## Partie 1 : Perceptron 

###### cette fonction permet de créer un reseau de neurones avec une couche d'entrée et une couche de sortie, elle a comme parametres les donnees d'entrés (image) et la variable cible (label) , ainsi que les poids initiaux du reseaux (avec la valeur constante du Biais) , le pas d'apprentissage et le nombre de neurone de la couche de sortie , qui correspond bien evidamment au nombre du classe du variable cible.

###### le processus de cette methode consiste principalement sur le calcul de l'activité pour chaque neurone du couche de sortie, puis on mesure l'erreur de prediction afin de faire la mise à jour des poids pour adpater les valeurs predites au valeurs réelles des classes.

In [31]:
def Single_Layer_Perceptron (image,label,weights,eta,output_nodes):
   
    x = image.numpy()[0]
    
    
    for i in range(0,output_nodes):
        activite = dot(x,weights[i][1:])+weights[i][0]  # weights[i][0]  : Biais
        y = label.numpy()[0][i]
        #print("y = ",y)
        error = y - activite
        #print("error = ",error)
        weights[i][1:] += eta * error * x
        weights[i][0] += error*eta 
        
    
    return weights




### Training phase

##### La phase d'apprentissage consiste à faire tourner ce processus sur plusieurs iterations afin de garantir la convergence des poids pour une meilleure classification.
##### après plusieurs essais sur les differents paramétres du notre reseau Perceptron, on a remarqué que pour le perceptron simple avec une seule couche, les parametres qui donnent une meilleure precision au niveau de classification correspond principalement sur une valeur de pas d'apprentissge très petite , en plus le nombre d'iteration pour l'apprentissage peut influencer aussi sur les poids de notre perceptron

In [32]:
inputs = 28*28
output_nodes = 10
eta = 0.0001

size = inputs*output_nodes
w = Weights(size,-0.25,0.25)
w = numpy.array(w).reshape(output_nodes, inputs)
w = numpy.insert(w, 0, 1, axis=1) # add Biais

nb_iteration = 100

for i in range(0,nb_iteration):
    for image,label in train_loader:
        w = Single_Layer_Perceptron(image,label,w,eta,output_nodes)



        

### Test phase

In [ ]:
    
for image in test_loader:
    
    x = image.numpy()[0]
    List_predictions = []
    for i in range(0,output_nodes):
        
        predicted = dot(x,w[i][1:])+w[i][0]
        List_predictions.append(predicted)
        
    y_pred = numpy.argmax(List_predictions)
    
    affichage2(image.numpy(),y_pred)
    
    
    

In [33]:
List_predictions = []
list_trueValues = []

counter = 0

for image in test_loader:
    
    x = image.numpy()[0]

    pred = []
    #print(x)
    for i in range(0,output_nodes):

        predicted = dot(x,w[i][1:])
        pred.append(predicted)
    
    y_pred = numpy.argmax(pred)
    #print(pred)
    list_trueValues.append(numpy.argmax(test_data_label[counter]))
    List_predictions.append(y_pred)
    counter = counter + 1
    if (counter == len(test_data_label)):
        break
        
    #print("predicted = ",y_pred , "  true = ",numpy.argmax(test_data_label[o]))
    
p = 0

# get values that were predicted right

for b in range(0,len(List_predictions)):
    if List_predictions[b] == list_trueValues[b]:
        p = p + 1

print("Accuracy  = ",p/7000)

Accuracy  =  0.8072857142857143


## Partie 2 : Shallow network
### Multi Layer Perceptron 

##### Pour la definition du perceptron multi couche, la fonction prend comme paramètres bien evidamment les données d'entrée et la variable cible, plus les poids de chacune des deux couches du perceptron : couche cachée et la couche de sortie , et le nombre de neurones de la couche cachée et la couche de sortie

##### On commence le procesus par presenter les donnees d'entrées pour effectuer apres la propagation de l'activité d'abord au niveau de la couche cachée qui va servir par la suite pour calculer l'activité du la couche de sortie. En revanche , l'étape de retro-propagation est faite pour calculer l'erreur pour les deux couches de perceptron, afin de faire la mise à jour de leurs poids en se basant sur l'erreur calculé du couche courante et sa donnée d'entrée.

In [34]:
def Multi_Layer_Perceptron (image,label,Hidden_Layer_weights,output_Layer_weights,eta,hidden_nodes,output_nodes):
    
    x = image.numpy()[0]
    output_nodes_errors = []
    
    output_nodes_sigmoidValues = []
    for k in range(0,output_nodes):
        
        y = label.numpy()[0][k]
        
        

        
        hidden_sigmoid_output = []
        
        for i in range(0,hidden_nodes):
            
            hidden_activite = dot(x,Hidden_Layer_weights[i][1:])+Hidden_Layer_weights[i][0]
            sig = sigmoid(hidden_activite)
            hidden_sigmoid_output.append(sig)
        
        output_nodes_sigmoidValues.append(hidden_sigmoid_output)
        
        output_activite = dot(hidden_sigmoid_output,output_Layer_weights[k][1:])+output_Layer_weights[k][0]
        output_error = y - output_activite
        output_nodes_errors.append(output_error)
        
    hidden_nodes_errors = []
    
    w22 = numpy.transpose(output_Layer_weights)
    w22 = numpy.delete(w22, 0, 0)
    
    allsigmo = numpy.transpose(output_nodes_sigmoidValues)

    for i in range(0,hidden_nodes):
        hidden_error = 0.0
        for j in range(0,output_nodes):
            
            hidden_error += allsigmo[i][j]*(1-allsigmo[i][j])*output_nodes_errors[j]*w22[i][j]
            
        hidden_nodes_errors.append(hidden_error)
        
        
        Hidden_Layer_weights[i][1:] += eta * hidden_error*x
        Hidden_Layer_weights[i][0] += hidden_error*eta
        
      
    for o in range(0,output_nodes):
        
        output_Layer_weights[o][1:] += eta * output_nodes_errors[o] * numpy.array(output_nodes_sigmoidValues[o])
        output_Layer_weights[o][0] += output_nodes_errors[o]*eta
                
            

             
            
    return Hidden_Layer_weights , output_Layer_weights

### Training phase

##### Pour l'apprentissage , on remarque que toujours l'importance des parametres qui influencent concretement au qualité de classification , meme si on laisse le pas d'apprentissage pas assez petit qui peut causer une convergence rapide des poids mais par ailleurs l'augmentation du nombre des neurones du la couche cachée plus le nombre d'iterations pour l'apprentissage servent au fur et  à mesure de donner une precision plus importante au niveau du classification.

In [35]:
inputs = 28*28
hidden_nodes = 60
output_nodes = 10
eta = 0.02


size1 = inputs*hidden_nodes
size2 = hidden_nodes*output_nodes

Hidden_Layer_weights = Weights(size1,-1,1)
Hidden_Layer_weights = numpy.array(Hidden_Layer_weights).reshape(hidden_nodes, inputs)
Hidden_Layer_weights = numpy.insert(Hidden_Layer_weights, 0, 1, axis=1)

output_Layer_weights = Weights(size2,-1,1)
output_Layer_weights = numpy.array(output_Layer_weights).reshape(output_nodes, hidden_nodes)
output_Layer_weights = numpy.insert(output_Layer_weights, 0, 1, axis=1)



#print("intial weights = ",w2)




for i in range(0,10):
    for image,label in train_loader:

        #affichage(image.numpy(),label.numpy())
        Hidden_Layer_weights , output_Layer_weights = Multi_Layer_Perceptron(image,label,Hidden_Layer_weights,output_Layer_weights,eta,hidden_nodes,output_nodes)  


#print("final weights = ",w1,w2)        
        
        

### Test phase

In [160]:
List_predictions = []
list_trueValues = []

counter = 0

for image in test_loader:
    
    x = image.numpy()[0]

    pred = []
    #print(x)
    for i in range(0,output_nodes):
        sigmo = []
        for j in range(0,hidden_nodes):
            activite1 = dot(x,Hidden_Layer_weights[j][1:])+Hidden_Layer_weights[j][0]
            sig = sigmoid(activite1)
            sigmo.append(sig)
        
        activite2 = dot(sigmo,output_Layer_weights[i][1:])+output_Layer_weights[i][0]
        pred.append(activite2)    
    
    y_pred = numpy.argmax(pred)
    #print(pred)
    list_trueValues.append(numpy.argmax(test_data_label[counter]))
    List_predictions.append(y_pred)
    counter = counter + 1
    if (counter == len(test_data_label)):
        break
    #affichage2(image.numpy(),y_pred)
    #print("predicted = ",y_pred , "  true = ",numpy.argmax(test_data_label[o]))
p = 0

for b in range(0,len(list_trueValues)):
    if List_predictions[b] == list_trueValues[b]:
        p = p + 1

print("Accuracy  = ",p/7000)

Accuracy  =  0.9188571428571428


## partie 3 : Deep Network
### Model and training phase

##### Le principe du definition du model torch consite à creer les differents couches du model (les couches cachéeset la couche de sortie) , alors la focntion prend comme parametres les donnees d'entrées , le nombre des couches cachées que vous voulez crée , plus une liste (hidden_layers_nodes) qui contient le nobre des neurones pour chaque couche cachée .
##### Dans ce cas là, on a utilisé lq fonction sigmoid vue le meilleur resultat qui donne par rapport aux autres focntions 

In [38]:
def torch_model(input_nodes,number_hidden_layers,hidden_layers_nodes , output_nodes):
    
    layers = []
    layers.append(torch.nn.Linear(input_nodes, hidden_layers_nodes[0]))
    layers.append(torch.nn.Sigmoid())
    for i in range(0,number_hidden_layers):

        layers.append(torch.nn.Linear(hidden_layers_nodes[i], hidden_layers_nodes[i+1]))
        layers.append(torch.nn.Sigmoid())
    layers.append(torch.nn.Linear(hidden_layers_nodes[len(hidden_layers_nodes)-1],output_nodes))  
    model = torch.nn.Sequential(*layers)
    
    return model

In [39]:
model = torch_model(28*28,2,[200,100,50],10)
model

Sequential(
  (0): Linear(in_features=784, out_features=200)
  (1): Sigmoid()
  (2): Linear(in_features=200, out_features=100)
  (3): Sigmoid()
  (4): Linear(in_features=100, out_features=50)
  (5): Sigmoid()
  (6): Linear(in_features=50, out_features=10)
)

### Training phase

##### on remarque que plus le nombre des couches cachées est grand ainsi que le nombre de neurones de chacune de ces couches est aussi grand , la précision du classification s'améliore en corrélation
##### aussi le nombre d'iteration pour l'apprentissage influence d'une manière importante à la precision  , meme si le pas d'apprentissage n'est pas petit 

In [40]:
import torch
from torch.autograd import Variable

Xtr=Variable(torch.stack([torch.Tensor(i) for i in train_data]))
Ytr=Variable(torch.stack([torch.Tensor(i) for i in train_data_label]))

Xte=Variable(torch.stack([torch.Tensor(i) for i in test_data]),requires_grad=False)
Yte=Variable(torch.stack([torch.Tensor(i) for i in test_data_label]),requires_grad=False)

learning_rate = 0.01
# Use the nn package to define our model and loss function.

loss_fn = torch.nn.MSELoss(size_average=False)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


for t in range(500):
    
  # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(Xtr)

  # Compute and print loss.
    loss = loss_fn(y_pred, Ytr)
    
    #print(y_pred)
  

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

### Test phase

In [41]:
pred = model(Xte)

In [42]:
pr = numpy.array(pred.data)
yt = numpy.array(Yte.data)
pr[8]

array([-0.02299722,  0.00442303,  0.00881158,  0.0063893 , -0.01643628,
        0.00310716, -0.00696279, -0.0080597 ,  0.00467176,  1.02625406], dtype=float32)

In [45]:
cn = 0 
for i in range(0,7000):
    
    pre = numpy.argmax(pr[i])
    
    ytrue = numpy.argmax(yt[i])
    
    if (pre == ytrue):
        cn = cn + 1
        
print("Accuracy  = ",cn/7000)

Accuracy  =  0.9728571428571429


## Partie 4 : Optionnel

### 1- Activation function

#### Tanh()

In [46]:
def torch_model(input_nodes,number_hidden_layers,hidden_layers_nodes , output_nodes):
    
    layers = []
    layers.append(torch.nn.Linear(input_nodes, hidden_layers_nodes[0]))
    layers.append(torch.nn.Tanh())
    for i in range(0,number_hidden_layers):

        layers.append(torch.nn.Linear(hidden_layers_nodes[i], hidden_layers_nodes[i+1]))
        layers.append(torch.nn.Tanh())
    layers.append(torch.nn.Linear(hidden_layers_nodes[len(hidden_layers_nodes)-1],output_nodes))  
    model = torch.nn.Sequential(*layers)
    
    return model

In [47]:
model = torch_model(28*28,2,[100,80,40],10)
model

Sequential(
  (0): Linear(in_features=784, out_features=100)
  (1): Tanh()
  (2): Linear(in_features=100, out_features=80)
  (3): Tanh()
  (4): Linear(in_features=80, out_features=40)
  (5): Tanh()
  (6): Linear(in_features=40, out_features=10)
)

### Training phase

In [48]:
import torch
from torch.autograd import Variable

Xtr=Variable(torch.stack([torch.Tensor(i) for i in train_data]))
Ytr=Variable(torch.stack([torch.Tensor(i) for i in train_data_label]))

Xte=Variable(torch.stack([torch.Tensor(i) for i in test_data]),requires_grad=False)
Yte=Variable(torch.stack([torch.Tensor(i) for i in test_data_label]),requires_grad=False)

learning_rate = 0.01
# Use the nn package to define our model and loss function.

loss_fn = torch.nn.MSELoss(size_average=False)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


for t in range(500):
    
  # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(Xtr)

  # Compute and print loss.
    loss = loss_fn(y_pred, Ytr)
    
    #print(y_pred)
  

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

### Test phase

In [54]:
pred = model(Xte)
pr = numpy.array(pred.data)
yt = numpy.array(Yte.data)
cn = 0 
for i in range(0,7000):
    
    pre = numpy.argmax(pr[i])
    
    ytrue = numpy.argmax(yt[i])
    if (pre == ytrue):
        cn = cn + 1
        
print("Accuracy  = ",cn/7000)

Accuracy  =  0.9717142857142858


#### ReLU()

### Model definition and Traning phase

In [55]:
def torch_model(input_nodes,number_hidden_layers,hidden_layers_nodes , output_nodes):
    
    layers = []
    layers.append(torch.nn.Linear(input_nodes, hidden_layers_nodes[0]))
    layers.append(torch.nn.ReLU())
    for i in range(0,number_hidden_layers):

        layers.append(torch.nn.Linear(hidden_layers_nodes[i], hidden_layers_nodes[i+1]))
        layers.append(torch.nn.ReLU())
    layers.append(torch.nn.Linear(hidden_layers_nodes[len(hidden_layers_nodes)-1],output_nodes))  
    model = torch.nn.Sequential(*layers)
    
    return model

model = torch_model(28*28,2,[200,100,50],10)


import torch
from torch.autograd import Variable

Xtr=Variable(torch.stack([torch.Tensor(i) for i in train_data]))
Ytr=Variable(torch.stack([torch.Tensor(i) for i in train_data_label]))

Xte=Variable(torch.stack([torch.Tensor(i) for i in test_data]),requires_grad=False)
Yte=Variable(torch.stack([torch.Tensor(i) for i in test_data_label]),requires_grad=False)

learning_rate = 0.01
# Use the nn package to define our model and loss function.

loss_fn = torch.nn.MSELoss(size_average=False)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


for t in range(200):
    
  # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(Xtr)

  # Compute and print loss.
    loss = loss_fn(y_pred, Ytr)
    
    #print(y_pred)
  

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
    

### Test phase

In [57]:
pred = model(Xte)
pr = numpy.array(pred.data)
yt = numpy.array(Yte.data)
cn = 0 
for i in range(0,7000):
    
    pre = numpy.argmax(pr[i])
    
    ytrue = numpy.argmax(yt[i])
    
    if (pre == ytrue):
        cn = cn + 1
        
print("Accuracy  = ",cn/7000)

Accuracy  =  0.968


### 2- Optimizer

#### Adagrad 

### Model definition and training phase

In [58]:
def torch_model(input_nodes,number_hidden_layers,hidden_layers_nodes , output_nodes):
    
    layers = []
    layers.append(torch.nn.Linear(input_nodes, hidden_layers_nodes[0]))
    layers.append(torch.nn.ReLU())
    for i in range(0,number_hidden_layers):

        layers.append(torch.nn.Linear(hidden_layers_nodes[i], hidden_layers_nodes[i+1]))
        layers.append(torch.nn.ReLU())
    layers.append(torch.nn.Linear(hidden_layers_nodes[len(hidden_layers_nodes)-1],output_nodes))  
    model = torch.nn.Sequential(*layers)
    
    return model

model = torch_model(28*28,2,[100,60,20],10)


import torch
from torch.autograd import Variable

Xtr=Variable(torch.stack([torch.Tensor(i) for i in train_data]))
Ytr=Variable(torch.stack([torch.Tensor(i) for i in train_data_label]))

Xte=Variable(torch.stack([torch.Tensor(i) for i in test_data]),requires_grad=False)
Yte=Variable(torch.stack([torch.Tensor(i) for i in test_data_label]),requires_grad=False)

learning_rate = 0.01
# Use the nn package to define our model and loss function.

loss_fn = torch.nn.MSELoss(size_average=False)

optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)


for t in range(300):
    
  # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(Xtr)

  # Compute and print loss.
    loss = loss_fn(y_pred, Ytr)
    
    #print(y_pred)
  

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
    

### Test phase

In [60]:
pred = model(Xte)
pr = numpy.array(pred.data)
yt = numpy.array(Yte.data)
cn = 0 
for i in range(0,7000):
    
    pre = numpy.argmax(pr[i])
    
    ytrue = numpy.argmax(yt[i])
    
    if (pre == ytrue):
        cn = cn + 1
        
print("Accuracy  = ",cn/7000)

Accuracy  =  0.9568571428571429


### 3- Convolutionnal Network 

In [29]:
def torch_model(input_nodes,number_hidden_layers,hidden_layers_nodes , output_nodes):
    
    layers = []
    layers.append(torch.nn.Conv2d(input_nodes, hidden_layers_nodes[0],2))
    layers.append(torch.nn.ReLU())
    for i in range(0,number_hidden_layers):

        layers.append(torch.nn.Conv2d(hidden_layers_nodes[i], hidden_layers_nodes[i+1],2))
        layers.append(torch.nn.ReLU())
    layers.append(torch.nn.Conv2d(hidden_layers_nodes[len(hidden_layers_nodes)-1],output_nodes,2))  
    model = torch.nn.Sequential(*layers)
    
    return model

model = torch_model(28*28,2,[80,60,20],10)


import torch
from torch.autograd import Variable

Xtr=Variable(torch.stack([torch.Tensor(i) for i in train_data]))
Ytr=Variable(torch.stack([torch.Tensor(i) for i in train_data_label]))

Xte=Variable(torch.stack([torch.Tensor(i) for i in test_data]),requires_grad=False)
Yte=Variable(torch.stack([torch.Tensor(i) for i in test_data_label]),requires_grad=False)

learning_rate = 0.01
# Use the nn package to define our model and loss function.

loss_fn = torch.nn.MSELoss(size_average=False)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


for t in range(200):
    
  # Forward pass: compute predicted y by passing x to the model.
    Xtr = Xtr.unsqueeze(0)
    y_pred = model(Xtr)

  # Compute and print loss.
    loss = loss_fn(y_pred, Ytr)
    
    #print(y_pred)
  

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
    

ValueError: Expected 4D tensor as input, got 3D tensor instead.

In [22]:
torch.nn.Conv2d?